<a href="https://colab.research.google.com/github/he16946/Data_Labs/blob/master/Lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/sagihaider/CE888_2020.git

Cloning into 'CE888_2020'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 25146 (delta 28), reused 4 (delta 0), pack-reused 25082
Receiving objects: 100% (25146/25146), 561.99 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [3]:
#Data
from zipfile import ZipFile
file_name = "/content/CE888_2020/Lab_8/data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [7]:
! pip install tensorflow==1.3.0
! pip install keras==2.0.7

     |████████████████████████████████| 43.6MB 92kB/s 
     |████████████████████████████████| 1.6MB 16.5MB/s 
     |████████████████████████████████| 890kB 43.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=071395ff467d2f929366e002f8c9c5b0b74a71ae3e4edcd420bd29253d6f2e9a
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: stable-baselines 2.2.1 has requirement tensorflow>=1.5.0, but you'll have tensorflow 1.3.0 which is incompatible.
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.3.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found existing installation: tensorflow 1.15.0
    Uninst

In [8]:
import numpy as np
import os
import time
from CE888_2020.Lab_8.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarn

In [11]:
#Load image 

img_path = '/content/CE888_2020/Lab_8/elephant.jpg' 
img = image.load_img(img_path, target_size = (224, 224)) #Loads an image into PIL format.
x = image.img_to_array(img) #Converts a PIL Image instance to a Numpy array.
print(x.shape)
x = np.expand_dims(x, axis=0) #number of dimensions increased by one
print(x.shape)
x = preprocess_input(x)
print('Input image shape', x.shape)

(224, 224, 3)
(1, 224, 224, 3)
Input image shape (1, 224, 224, 3)


In [0]:
PATH = os.getcwd() #returns current working directory of a process
#print(PATH)
data_path = PATH + '/data' 
data_dir_list = os.listdir(data_path) #Return a list containing the names of the entries in the directory given by path

In [17]:
img_data_list =[]

for dataset in data_dir_list:
  img_list=os.listdir(data_path+'/'+dataset)
  print('Loaded the images of dataset-'+'{}\n'.format(dataset))
  for img in img_list:
    img_path = data_path + '/' + dataset +'/' + img
    img = image.load_img(img_path,target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis =  0)
    #x = x/255
    #print('Input image shape:', x.shape) same as elephant example above
    img_data_list.append(x)

img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print(img_data.shape)
img_data = np.rollaxis(img_data,1,0) #roll aixs 1 to axis 0
print(img_data.shape)
img_data = img_data[0]
print(img_data.shape)

Loaded the images of dataset-Humans

Loaded the images of dataset-dogs

Loaded the images of dataset-horses

Loaded the images of dataset-cats

(808, 1, 224, 224, 3)
(1, 808, 224, 224, 3)
(808, 224, 224, 3)


In [20]:
num_classes = 4 # horses, humans, cats, dogs
print(img_data.shape[0])
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,), dtype='int64')

# in the data.zip file extracted in [2] the images are sorted cats>dogs>horses>humans
# with 202 pictures in each category

labels[0:202] = 0
labels[202:404] = 1
labels[404:606] = 2
labels[606:808] = 3

names = ['cats', 'dogs', 'horses', 'humans']

#convert to one_hot
Y = np_utils.to_categorical(labels, num_classes)

# shuffle
x,y = shuffle(img_data, Y, random_state=2) #shuffle sparse dataset in a consistent way - similar to resampling 
# split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

808


In [21]:
image_input = Input(shape=(224,224,3)) # shape of single image

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet') # include the 3 fully-connected layers at the top of the network with pre-learned weights
model.summary()

552591360/553467096 [============================>.] - ETA: 0s_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (M

In [23]:
last_layer = model.get_layer('fc2').output # retrieves the output of fc2
out = Dense(num_classes, activation= 'softmax', name='output')(last_layer) # replace 'predictions' with 'output'
custom_vgg_model = Model(image_input, out) 
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
for layer in custom_vgg_model.layers[:-1]:   #excluding last layer
   layer.trainable = False

custom_vgg_model.layerrs[3].trainable
custom_vgg_model.summary()

custom_vgg_model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
